In [1]:
import openai
from pathlib import Path
import json
import llm2geneset
import time
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer
from statsmodels.stats.multitest import multipletests

aclient = openai.AsyncClient()
client = openai.Client()

In [2]:
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2'], use_stemmer=True)

In [3]:
def flatten_list_of_lists(lst):
    return [item for sublist in lst for item in sublist]

def unflatten_list_of_lists(flat_list, original_structure):
    result = []
    index = 0
    for sublist in original_structure:
        length = len(sublist)
        result.append(flat_list[index:index + length] if length > 0 else [])
        index += length
    return result

In [4]:
lib_names = ["KEGG_2021_Human", 
             "Reactome_2022", 
             "WikiPathway_2023_Human"]
models = ["gpt-4o-mini-2024-07-18", "gpt-3.5-turbo-0125", "gpt-4o-2024-08-06"]

for lib_name in lib_names:
    print(lib_name)
    output = []
    for model in models:
        print(model)
        with open("libs_human/" + model + "/" + lib_name + ".json") as f:
            gen_res = json.load(f)
        
        test_sets = gen_res["curated_genesets"]
        test_descr = gen_res["descr"]
        test_descr_cleaned = gen_res["descr_cleaned"]

        gt_embs = llm2geneset.get_embeddings(client, test_descr_cleaned)

        for method in ["llm2geneset", "GSAI"]:
            if method == "GSAI":
                # use GSAI to generate geneset name
                gsai_res = await llm2geneset.gsai_bench(aclient, test_sets, model=model, n_retry=5)
                # use lists of size 1 for names.
                names = [[i['name']]for i in gsai_res]
                in_toks = [i["in_toks"] for i in gsai_res]
                out_toks = [i["out_toks"] for i in gsai_res]
            elif method == "llm2geneset":
                llm2geneset_res = await llm2geneset.gs_proposal_bench(aclient, test_sets, model=model)
                # Extract multiple names meeting significance.
                def res2name(res):
                    df = res["ora_results"]
                    _, pvals_corrected, _, _ = multipletests(df["p_val"], method='bonferroni')
                    df["p_adj"] = pvals_corrected
                    df = df[df["p_adj"] < 0.01]
                    if df.shape[0] == 0:
                        return ["None found."]
                    else: 
                        return df["set_descr"].to_list()            
                names = list(map(res2name, llm2geneset_res))
                in_toks = [i["tot_in_toks"] for i in llm2geneset_res]
                out_toks = [i["tot_out_toks"] for i in llm2geneset_res]

            # Since there can be multiple names per gene set
            # compute cosine similarity for each one. 
            flat_names = flatten_list_of_lists(names)
            flat_name_embs = llm2geneset.get_embeddings(client, flat_names)
            name_emb = unflatten_list_of_lists(flat_name_embs,names)

            # Get best score for significant gene sets and report. 
            for i, ref in enumerate(test_descr_cleaned):
                scores = [scorer.score(ref, n) for n in names[i]] 
                rouge1 = [s['rouge1'].recall for s in scores]
                rouge2 = [s['rouge2'].recall for s in scores]

                csim = [np.dot(gt_embs[i], emb) for emb in name_emb[i]]
                    
                x={"model":model,
                   "library":lib_name,
                   "gt_name":test_descr[i],
                   "gt_name_clean":ref,
                   "name":", ".join(names[i]),
                   "ROUGE1":max(rouge1),
                   "ROUGE2":max(rouge2),  
                   "csim":max(csim),
                   "method": method,
                   "in_toks":in_toks[i],
                   "out_toks": out_toks[i]                   
                  }
                output.append(x)
    # generate an output per library            
    df = pd.DataFrame(output) 
    outfile = "outputs/gsai_vs_llm2geneset_outputs_" + lib_name + ".tsv"
    df.to_csv(outfile, sep="\t", index=False)
      
        

Reactome_2022
gpt-4o-mini-2024-07-18


 73%|█████████████████████████████████████████████████████████████████████████████████████████▌                                | 1334/1818 [01:15<00:38, 12.66it/s]

retrying
List 5 biological pathways, biological processes, or cellular components that contain the following genes """KL,FGFBP1,FGFBP2,RBFOX2,FGFBP3,NCBP2,GALNT3,NCBP1,PTPN11,FRS3,PPP2R1A,BRAF,FRS2,GTF2F1,GTF2F2,HNRNPM,PIK3CA,ESRP2,ESRP1,HNRNPF,SOS1,KLB,SHC1,ANOS1,PTBP1,NRAS,TIAL1,FLRT3,MKNK1,FLRT1,FLRT2,FGF20,RPS27A,FGF23,FGF22,TIA1,PPP2CA,FGF18,FGF17,FGF16,HNRNPH1,PIK3R1,FGF19,FGF10,FGFR1,SPRY2,GRB2,UBA52,FGF13,FGFR4,FGFR3,FGFR2,SRC,PIGU,FGF1,CBL,FGF3,FGFRL1,FGF4,FGF5,FGF6,PPP2CB,FGF7,FGF8,SPRED2,FGF9,POLR2A,SPRED1,POLR2B,UBC,POLR2C,UBB,POLR2D,POLR2E,MAPK3,POLR2F,POLR2G,HRAS,MAPK1,PLCG1,POLR2H,POLR2I,POLR2J,POLR2K,POLR2L,HNRNPA1""" with high confidence. Be as specific as possible. List non-overlapping pathways, processes, or components. Do not include the gene names in the outputs. Use the following JSON schema:
```json
{
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "p": {
                "type": "string",
            },
        },

  2%|██▍                                                                                                                         | 35/1818 [00:07<02:35, 11.45it/s]

retrying
name is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

...

 59%|███████████████████████████████████████████████████████████████████████▍                                                  | 1065/1818 [01:09<00:27, 27.51it/s]

retrying
conf is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 1801/1818 [01:49<00:03,  4.67it/s]

retrying
name is none
Write a critical analysis of the biological processes performed by this system of interacting proteins.

Base your analysis on prior knowledge available in your training data.  After completing your analysis, propose a brief and
detailed name for the most prominent biological process performed by the system.

After completing your analysis, please also assign a confidence score to the process name you selected.  This score should
follow the name in parentheses and range from 0.00 to 1.00. A score of 0.00 indicates the lowest confidence, while 1.00 reflects
the highest confidence. This score helps gauge how accurately the chosen name represents the functions and activities within
the system of interacting proteins. When determining your score, consider the proportion of genes in the protein system that
participate in the identified biological process. For instance, if you select "Ribosome biogenesis" as the process name but
only a few genes in the system contribute

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1818/1818 [02:01<00:00, 14.93it/s]
